# Handwritten Chinese OCR - Colab Training

本 Notebook 用于在 Google Colab 上训练手写中文 OCR 模型 (CASIA-HWDB2.0 数据集)。

## 使用步骤
1. 将整个项目文件夹（包含 `HWDB2.0Train` 和 `HWDB2.0Test`）上传到 Google Drive
2. 在 Colab 中打开此 Notebook，选择 GPU 运行时
3. 按顺序运行每个 Cell

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive already mounted")
else:
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully")

## 2. Set Project Path

**⚠️ Modify `PROJECT_PATH` below to match your Google Drive folder location.**

In [ ]:
# ⚠️ MODIFY THIS PATH to your project folder in Google Drive
PROJECT_PATH = '/content/drive/MyDrive/handwritten-chinese-ocr-samples'

# Change to project directory
%cd {PROJECT_PATH}

# Verify project structure
!ls -la
print("\n" + "="*50)
print(f"✅ Working directory: {os.getcwd()}")

## 3. Install Dependencies

In [ ]:
# Install dependencies from requirements.txt
!pip install -q -r requirements.txt

# Verify PyTorch installation
import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 3.5 Update Code Files (Important Fix!)

**⚠️ The original model had a CTC bug causing Loss=0.0000 and no learning.**

The fix: Reduced maxpool layers from 5→4 so the output sequence length is long enough for CTC loss to work with Chinese text labels (40+ characters).

Run the cell below to download the fixed files from GitHub:

In [ ]:
# Download fixed files from GitHub
import urllib.request
import os

base_url = "https://raw.githubusercontent.com/AndrewCullacino/handwritten-chinese-ocr-samples/main/"

files_to_update = [
    "models/handwritten_ctr_model.py",  # Fixed: 4 maxpool instead of 5
    "main.py",                           # Added CTC debug logging
]

print("Updating code files from GitHub...")
print("="*50)

for f in files_to_update:
    url = base_url + f
    dest = f
    os.makedirs(os.path.dirname(dest) if os.path.dirname(dest) else '.', exist_ok=True)
    urllib.request.urlretrieve(url, dest)
    print(f"✅ Updated: {f}")

# Delete old checkpoints (incompatible with new model architecture)
import glob
old_models = glob.glob('hctr_*.pth.tar')
if old_models:
    print(f"\n⚠️ Found {len(old_models)} old model checkpoints.")
    print("   These are incompatible with the fixed model (different layer sizes).")
    confirm = input("   Delete old checkpoints? (y/n): ").strip().lower()
    if confirm == 'y':
        for f in old_models:
            os.remove(f)
            print(f"   🗑️ Deleted: {f}")
    else:
        print("   ⚠️ Keep old models. Training will fail if you try to resume from them.")

print("\n✅ Code update complete! Now run training.")

## 4. Data Preparation Notes

### ⚠️ IMPORTANT: Dataset Preprocessing Required

Before training, the raw **CASIA-HWDB2.x** binary files (`.dgrl`, `.dgr`, `.gnt` formats) must be preprocessed into image files (PNG) and label files.

### Preprocessing Utilities

The following utility scripts are available in `utils/casia-hwdb-data-preparation/`:

| Script | Purpose | Input Format |
|--------|---------|-------------|
| `preprocess_dgrl.py` | Convert DGRL text line files to PNG + labels | `.dgrl` |
| `dgr2png.c` | C/C++ utility for converting DGR files | `.dgr` |
| `gnt2png.py` | Convert GNT character files to PNG | `.gnt` |

### Expected Dataset Structure After Preprocessing

```
data/hwdb2.0/
├── train/
│   ├── 000000.png
│   ├── 000001.png
│   └── ...
├── val/
│   └── ...
├── test/
│   └── ...
├── train_img_id_gt.txt    # Format: image_name,label_text
├── val_img_id_gt.txt
├── test_img_id_gt.txt
└── chars_list.txt         # One character per line
```

### Example: Preprocessing DGRL Files

```bash
# Run from project root
python preprocess_dgrl.py \
    --train_dir HWDB2.0Train \
    --test_dir HWDB2.0Test \
    --output_dir data/hwdb2.0 \
    --val_split 0.1
```

### Compiling dgr2png.c (if needed)

```bash
cd utils/casia-hwdb-data-preparation
gcc -o dgr2png dgr2png.c
./dgr2png <input.dgr> <output_dir>
```

## 5. Verify Dataset

In [ ]:
# Dataset path (modify if using a different location)
DATASET_PATH = 'data/hwdb2.0'  # or 'data/handwritten_ctr_data'

# Check dataset files
import os

required_files = [
    f'{DATASET_PATH}/train_img_id_gt.txt',
    f'{DATASET_PATH}/val_img_id_gt.txt', 
    f'{DATASET_PATH}/test_img_id_gt.txt',
    f'{DATASET_PATH}/chars_list.txt',
    f'{DATASET_PATH}/train',
    f'{DATASET_PATH}/val',
    f'{DATASET_PATH}/test'
]

print("Dataset verification:")
print("="*50)
all_ok = True
for f in required_files:
    exists = os.path.exists(f)
    status = "✅" if exists else "❌"
    print(f"{status} {f}")
    if not exists:
        all_ok = False

if all_ok:
    # Count samples
    for split in ['train', 'val', 'test']:
        gt_file = f'{DATASET_PATH}/{split}_img_id_gt.txt'
        with open(gt_file, 'r', encoding='utf-8') as f:
            count = len(f.readlines())
        print(f"   {split}: {count} samples")
    
    with open(f'{DATASET_PATH}/chars_list.txt', 'r', encoding='utf-8') as f:
        num_chars = len(f.readlines())
    print(f"   Character vocabulary: {num_chars}")
else:
    print("\n⚠️ Some files are missing. Please preprocess the dataset first.")
    print("   See 'Data Preparation Notes' section above.")

## 6. Training

In [ ]:
# Training configuration
DATASET_PATH = 'data/hwdb2.0'  # Path to preprocessed dataset
BATCH_SIZE = 8                  # Adjust based on GPU memory
EPOCHS = 30                     # Recommended: 30+ epochs for good CER
PRINT_FREQ = 50                 # Print frequency
NUM_WORKERS = 2                 # Data loading workers

# Run training
# You should see:
#   - [CTC Debug] showing output seq_len > max target len (GOOD!)
#   - Loss > 0 and decreasing (model is learning!)
#   - CER decreasing over epochs
!python main.py -m hctr \
    -d {DATASET_PATH} \
    -b {BATCH_SIZE} \
    -ep {EPOCHS} \
    -pf {PRINT_FREQ} \
    -j {NUM_WORKERS}

## 7. Find Best Model

In [ ]:
import glob

# Find saved model files
model_files = glob.glob('hctr_*.pth.tar')

if model_files:
    print("Saved models:")
    for f in sorted(model_files):
        size_mb = os.path.getsize(f) / (1024*1024)
        print(f"  📁 {f} ({size_mb:.1f} MB)")
    
    # Select best model (with 'acc' in name, or latest checkpoint)
    acc_models = [f for f in model_files if 'acc' in f]
    if acc_models:
        BEST_MODEL = sorted(acc_models)[-1]
    else:
        BEST_MODEL = 'hctr_checkpoint.pth.tar'
    
    print(f"\n✅ Selected model: {BEST_MODEL}")
else:
    print("❌ No model files found. Please run training first.")
    BEST_MODEL = None

## 8. Evaluation

In [ ]:
# Evaluation configuration
DATASET_PATH = 'data/hwdb2.0'
MODEL_FILE = BEST_MODEL  # Or specify manually: 'hctr_checkpoint.pth.tar'
TEST_PATH = f'{DATASET_PATH}/test'
BATCH_SIZE = 16

if MODEL_FILE and os.path.exists(MODEL_FILE):
    print(f"Evaluating model: {MODEL_FILE}")
    print(f"Test set: {TEST_PATH}")
    print("="*50)
    
    # Run evaluation with benchmark mode (-bm)
    !python test.py -m hctr \
        -f {MODEL_FILE} \
        -i {TEST_PATH} \
        -b {BATCH_SIZE} \
        -bm \
        -dm greedy-search \
        -pf 20
else:
    print("❌ Model file not found. Please run training first or specify the correct path.")

## 9. Sample Predictions

In [ ]:
import torch
import cv2
import random
import matplotlib.pyplot as plt
from models.handwritten_ctr_model import hctr_model
from utils.ctc_codec import ctc_codec
from utils.dataset import NormalizePAD

DATASET_PATH = 'data/hwdb2.0'

if MODEL_FILE and os.path.exists(MODEL_FILE):
    # Load model
    model = hctr_model()
    checkpoint = torch.load(MODEL_FILE, map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    
    # Load character set
    with open(f'{DATASET_PATH}/chars_list.txt', 'r', encoding='utf-8') as f:
        characters = ''.join([line.strip() for line in f])
    codec = ctc_codec(characters)
    
    # Load test labels
    test_labels = {}
    with open(f'{DATASET_PATH}/test_img_id_gt.txt', 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(',', 1)
            if len(parts) == 2:
                test_labels[parts[0]] = parts[1]
    
    # Random samples
    samples = random.sample(list(test_labels.keys()), min(4, len(test_labels)))
    
    fig, axes = plt.subplots(len(samples), 1, figsize=(15, 3*len(samples)))
    if len(samples) == 1:
        axes = [axes]
    
    for ax, img_name in zip(axes, samples):
        img_path = f'{DATASET_PATH}/test/{img_name}'
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Predict
        h, w = img.shape
        img_tensor = img[:, :, None]
        transform = NormalizePAD((1, 128, w))
        img_tensor = transform(img_tensor).unsqueeze(0)
        
        with torch.no_grad():
            preds = model(img_tensor)
            result = codec.decode(preds.cpu().numpy())[0]
        
        true_label = test_labels[img_name]
        match = "✅" if result == true_label else "❌"
        
        ax.imshow(img, cmap='gray')
        ax.set_title(f"{match} GT: {true_label}\nPred: {result}", fontsize=10, loc='left')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ Model not available for prediction.")

## 10. Save Model to Drive

In [ ]:
import shutil

# Create checkpoints directory in Drive
SAVE_DIR = f'{PROJECT_PATH}/checkpoints'
os.makedirs(SAVE_DIR, exist_ok=True)

# Copy all model files
model_files = glob.glob('hctr_*.pth.tar')
for f in model_files:
    dst = os.path.join(SAVE_DIR, f)
    shutil.copy2(f, dst)
    print(f"✅ Saved: {dst}")

print(f"\n🎉 All models saved to: {SAVE_DIR}")

## 1. 检查 GPU

In [ ]:
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 挂载 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 设置路径并复制项目

⚠️ **修改下面的路径为你 Google Drive 中的实际位置**

In [ ]:
# ========== 修改这里 ==========
DRIVE_PROJECT = '/content/drive/MyDrive/handwritten-chinese-ocr-samples'
# ==============================

# 查看 Drive 根目录帮助定位
!ls /content/drive/MyDrive/ | head -20

In [ ]:
# 复制项目到 Colab 本地 (更快的读写速度)
!cp -r "{DRIVE_PROJECT}" /content/
%cd /content/handwritten-chinese-ocr-samples
!ls -la

## 4. 安装依赖

In [ ]:
!pip install -q editdistance
print("✅ 依赖安装完成")

## 5. 创建 DGRL 预处理脚本

这是修复后的脚本，可以正确解析 CASIA-HWDB2.0 的 DGRL 格式

In [ ]:
%%writefile preprocess_dgrl.py
"""
DGRL Preprocessor for CASIA-HWDB2.x
Converts .dgrl files to PNG images + labels for OCR training.
"""

import os
import struct
import numpy as np
from PIL import Image
import argparse


def parse_dgrl_file(filepath, debug=False):
    """Parse a single DGRL file and extract text lines with images."""
    lines_data = []
    
    try:
        with open(filepath, 'rb') as f:
            data = f.read()
        
        if len(data) < 85:
            return lines_data
        
        # Header: 73 bytes, then page info
        pos = 73
        page_w = struct.unpack('<I', data[pos:pos+4])[0]
        pos += 4
        page_h = struct.unpack('<I', data[pos:pos+4])[0]
        pos += 4
        num_lines = struct.unpack('<I', data[pos:pos+4])[0]
        pos += 4
        
        if debug:
            print(f"  Page: {page_w}x{page_h}, {num_lines} lines")
        
        if num_lines > 50 or num_lines == 0:
            return lines_data
        
        for line_idx in range(num_lines):
            if pos + 4 > len(data):
                break
            
            num_chars = struct.unpack('<I', data[pos:pos+4])[0]
            pos += 4
            
            if debug:
                print(f"  Line {line_idx}: num_chars={num_chars}")
            
            if num_chars > 200 or num_chars == 0:
                break
            
            # Read GB codes (2 bytes each)
            text = ''
            for _ in range(num_chars):
                if pos + 2 > len(data):
                    break
                code = data[pos:pos+2]
                pos += 2
                try:
                    char = code.decode('gb18030')
                    text += char
                except:
                    pass
            
            if pos + 8 > len(data):
                break
            
            # Image dimensions
            height = struct.unpack('<I', data[pos:pos+4])[0]
            pos += 4
            width = struct.unpack('<I', data[pos:pos+4])[0]
            pos += 4
            
            if debug:
                print(f"    Image: {width}x{height}, text: {text[:20]}...")
            
            if height > 2000 or width > 8000 or height == 0 or width == 0:
                break
            
            img_size = height * width
            if pos + img_size > len(data):
                break
            
            img_array = np.frombuffer(data[pos:pos+img_size], dtype=np.uint8)
            img_array = img_array.reshape(height, width)
            pos += img_size
            
            # KEY: Skip trailing line data until 0xFF
            while pos < len(data) and data[pos] != 0xFF:
                pos += 1
            
            # Skip 0xFF padding
            while pos < len(data) and data[pos] == 0xFF:
                pos += 1
            
            if text:
                lines_data.append({
                    'image': img_array,
                    'text': text,
                    'height': height,
                    'width': width
                })
                
    except Exception as e:
        if debug:
            print(f"  Error: {e}")
    
    return lines_data


def process_directory(input_dir, output_dir, phase, target_height=128):
    """Process all DGRL files in a directory."""
    os.makedirs(os.path.join(output_dir, phase), exist_ok=True)
    
    dgrl_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.dgrl')])
    print(f"Found {len(dgrl_files)} DGRL files in {input_dir}")
    
    all_pairs = []
    all_chars = set()
    
    for i, dgrl_file in enumerate(dgrl_files):
        filepath = os.path.join(input_dir, dgrl_file)
        lines = parse_dgrl_file(filepath)
        
        for line_idx, line_data in enumerate(lines):
            img = line_data['image']
            h, w = img.shape
            ratio = target_height / h
            new_w = max(1, int(w * ratio))
            
            pil_img = Image.fromarray(img)
            pil_img = pil_img.resize((new_w, target_height), Image.Resampling.LANCZOS)
            
            img_name = f"{dgrl_file.replace('.dgrl', '')}_{line_idx:03d}.png"
            pil_img.save(os.path.join(output_dir, phase, img_name))
            
            all_pairs.append((img_name, line_data['text']))
            all_chars.update(line_data['text'])
        
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{len(dgrl_files)}, {len(all_pairs)} lines")
    
    gt_file = os.path.join(output_dir, f'{phase}_img_id_gt.txt')
    with open(gt_file, 'w', encoding='utf-8') as f:
        for img_name, text in all_pairs:
            f.write(f'{img_name},{text}\n')
    
    print(f"{phase}: {len(all_pairs)} lines extracted")
    return all_chars


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_dir', default='HWDB2.0Train')
    parser.add_argument('--test_dir', default='HWDB2.0Test')
    parser.add_argument('--output_dir', default='data/hwdb2.0')
    parser.add_argument('--val_split', type=float, default=0.1)
    parser.add_argument('--target_height', type=int, default=128)
    args = parser.parse_args()
    
    os.makedirs(args.output_dir, exist_ok=True)
    all_chars = set()
    
    # Process training data
    if os.path.isdir(args.train_dir):
        print("Processing training data...")
        chars = process_directory(args.train_dir, args.output_dir, 'train', args.target_height)
        all_chars.update(chars)
        
        # Split validation
        train_gt = os.path.join(args.output_dir, 'train_img_id_gt.txt')
        if os.path.exists(train_gt):
            with open(train_gt, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            import random
            random.seed(42)
            random.shuffle(lines)
            
            val_size = int(len(lines) * args.val_split)
            val_lines, train_lines = lines[:val_size], lines[val_size:]
            
            val_dir = os.path.join(args.output_dir, 'val')
            train_dir = os.path.join(args.output_dir, 'train')
            os.makedirs(val_dir, exist_ok=True)
            
            for line in val_lines:
                img_name = line.strip().split(',')[0]
                src = os.path.join(train_dir, img_name)
                dst = os.path.join(val_dir, img_name)
                if os.path.exists(src):
                    os.rename(src, dst)
            
            with open(train_gt, 'w', encoding='utf-8') as f:
                f.writelines(train_lines)
            with open(os.path.join(args.output_dir, 'val_img_id_gt.txt'), 'w', encoding='utf-8') as f:
                f.writelines(val_lines)
            
            print(f"Split: {len(train_lines)} train, {len(val_lines)} val")
    
    # Process test data
    if os.path.isdir(args.test_dir):
        print("Processing test data...")
        chars = process_directory(args.test_dir, args.output_dir, 'test', args.target_height)
        all_chars.update(chars)
    
    # Save char list
    with open(os.path.join(args.output_dir, 'chars_list.txt'), 'w', encoding='utf-8') as f:
        for char in sorted(all_chars):
            f.write(char + '\n')
    
    print(f"\nDone! {len(all_chars)} unique characters")
    print(f"Output: {args.output_dir}")


if __name__ == '__main__':
    main()

## 6. 测试预处理脚本

In [ ]:
# 测试解析单个文件
import os
from preprocess_dgrl import parse_dgrl_file

dgrl_files = sorted([f for f in os.listdir('HWDB2.0Train') if f.endswith('.dgrl')])
print(f"Found {len(dgrl_files)} dgrl files")

test_file = f'HWDB2.0Train/{dgrl_files[0]}'
print(f"\nTesting: {test_file}")
lines = parse_dgrl_file(test_file, debug=True)
print(f"\n✅ Extracted {len(lines)} lines")

## 7. 运行完整预处理 ⏰

这一步需要 10-20 分钟

In [ ]:
# 删除旧数据（如果有）
!rm -rf data/hwdb2.0

# 运行预处理
!python preprocess_dgrl.py --train_dir HWDB2.0Train --test_dir HWDB2.0Test --output_dir data/hwdb2.0

## 8. 检查数据集

In [ ]:
print("=== 数据集统计 ===")
!wc -l data/hwdb2.0/*_img_id_gt.txt

print("\n=== 样本示例 ===")
!head -3 data/hwdb2.0/train_img_id_gt.txt

print("\n=== 字符表大小 ===")
!wc -l data/hwdb2.0/chars_list.txt

In [ ]:
# 可视化样本
import matplotlib.pyplot as plt
from PIL import Image
import os

with open('data/hwdb2.0/train_img_id_gt.txt', 'r', encoding='utf-8') as f:
    samples = [line.strip().split(',', 1) for line in f.readlines()[:3]]

fig, axes = plt.subplots(3, 1, figsize=(15, 5))
for i, (img_name, text) in enumerate(samples):
    img_path = os.path.join('data/hwdb2.0/train', img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(text[:40] + '...' if len(text) > 40 else text)
        axes[i].axis('off')
plt.tight_layout()
plt.show()

## 9. 开始训练 🚀

In [ ]:
# 训练模型
# -b 16: batch size (可根据 GPU 内存调整)
# -ep 30: 训练 30 轮
# -pf 100: 每 100 步打印一次
!python main.py -m hctr -d data/hwdb2.0 -b 16 -pf 100 -ep 30 --gpu 0

## 10. 保存到 Google Drive ⚠️

Colab 会话可能随时断开，及时保存模型！

In [ ]:
# 保存 checkpoint 到 Drive
!mkdir -p /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints
!cp hctr_checkpoint.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/
!cp hctr_*acc*.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/ 2>/dev/null || true

# 保存预处理数据 (下次可跳过预处理)
!cp -r data/hwdb2.0 /content/drive/MyDrive/handwritten-chinese-ocr/

print("✅ 已保存到 Google Drive!")

## 11. 测试模型

In [ ]:
# 在测试集上评估
!python test.py -m hctr -f hctr_checkpoint.pth.tar -i data/hwdb2.0 -bm -dm greedy-search --gpu 0

## 12. 恢复训练 (可选)

如果 Colab 断开，取消下面的注释恢复训练

In [ ]:
# # 复制保存的数据和 checkpoint
# !cp -r /content/drive/MyDrive/handwritten-chinese-ocr/hwdb2.0 data/
# !cp /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/hctr_checkpoint.pth.tar .

# # 继续训练
# !python main.py -m hctr -d data/hwdb2.0 -b 16 -pf 100 -ep 50 --gpu 0 -re hctr_checkpoint.pth.tar

## 1. 检查 GPU 状态和 Python 版本

In [ ]:
!nvidia-smi
import sys
import torch

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 挂载 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 复制项目到 Colab 本地存储

⚠️ **请修改下面路径为你 Google Drive 中的实际位置**

In [ ]:
# ======= 请修改此路径 =======
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/handwritten-chinese-ocr-samples"
# ============================

# 复制到 Colab 本地磁盘（读写速度更快）
!cp -r {DRIVE_PROJECT_PATH} /content/
%cd /content/handwritten-chinese-ocr-samples

# 查看项目文件
!ls -la

## 4. 安装依赖

In [ ]:
!pip install -q editdistance
print("✅ 依赖安装完成!")

## 5. 预处理 DGRL 数据集 (首次运行)

将 CASIA-HWDB2.0 的 `.dgrl` 文件转换为 PNG 图片 + 标签文件。

⚡ **此步骤耗时较长 (~10-20分钟)，但只需运行一次**

In [ ]:
import os
import gc

# 检查是否已经预处理过
if os.path.exists('data/hwdb2.0/train_img_id_gt.txt'):
    print("✅ 数据已预处理，跳过此步骤")
    !wc -l data/hwdb2.0/*_img_id_gt.txt
else:
    print("🔄 开始预处理 DGRL 文件 (这需要 10-20 分钟)...")
    
    # 启用垃圾回收优化内存
    gc.enable()
    
    !python preprocess_dgrl.py \
        --train_dir HWDB2.0Train \
        --test_dir HWDB2.0Test \
        --output_dir data/hwdb2.0
    
    print("\n✅ 预处理完成!")

## 6. 检查数据集

In [ ]:
# 查看数据集统计
print("=== 数据集统计 ===")
!wc -l data/hwdb2.0/*_img_id_gt.txt

print("\n=== 样本示例 ===")
!head -3 data/hwdb2.0/train_img_id_gt.txt

print("\n=== 字符表大小 ===")
!wc -l data/hwdb2.0/chars_list.txt

In [ ]:
# 可视化样本
import matplotlib.pyplot as plt
from PIL import Image
import os

with open('data/hwdb2.0/train_img_id_gt.txt', 'r', encoding='utf-8') as f:
    samples = [line.strip().split(',', 1) for line in f.readlines()[:3]]

fig, axes = plt.subplots(3, 1, figsize=(15, 5))
for i, (img_name, text) in enumerate(samples):
    img_path = os.path.join('data/hwdb2.0/train', img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(text[:40] + '...' if len(text) > 40 else text)
        axes[i].axis('off')
plt.tight_layout()
plt.show()

## 7. 开始训练 🚀

**参数说明:**
- `-m hctr`: 手写中文文本识别模型
- `-d`: 数据集路径  
- `-b`: batch size (T4 GPU 建议 8-16)
- `-ep`: 训练轮数
- `-pf`: 打印频率
- `--gpu 0`: 使用 GPU

In [ ]:
# 训练模型 (完整数据集约需 2-4 小时/epoch)
!python main.py -m hctr -d data/hwdb2.0 -b 16 -pf 100 -ep 30 --gpu 0

## 8. 保存检查点到 Google Drive ⚠️

**重要**: Colab 会话可能随时断开，定期保存模型到 Drive!

In [ ]:
# 创建保存目录
!mkdir -p /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints

# 保存所有 checkpoint
!cp hctr_checkpoint.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/
!cp hctr_*acc*.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/ 2>/dev/null || true

# 保存预处理好的数据 (方便下次直接使用)
!cp -r data/hwdb2.0 /content/drive/MyDrive/handwritten-chinese-ocr/

print("✅ 模型和数据已保存到 Google Drive!")

## 9. 测试模型

In [ ]:
# 在测试集上评估
!python test.py -m hctr -f hctr_checkpoint.pth.tar -i data/hwdb2.0 -bm -dm greedy-search --gpu 0

## 10. 从断点恢复训练 (可选)

如果 Colab 断开，可以从 Drive 恢复继续训练：

In [ ]:
# 取消注释以下代码来恢复训练

# 如果之前已预处理数据，可直接复制
# !cp -r /content/drive/MyDrive/handwritten-chinese-ocr/hwdb2.0 data/

# 复制 checkpoint
# !cp /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/hctr_checkpoint.pth.tar .

# 继续训练
# !python main.py -m hctr -d data/hwdb2.0 -b 16 -pf 100 -ep 50 --gpu 0 -re hctr_checkpoint.pth.tar

## 1. 检查 GPU 状态

## 1. 检查 GPU 状态

In [ ]:
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 挂载 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 复制项目到 Colab 运行环境

**重要**: 请修改下面的路径为你实际上传到 Google Drive 的位置

In [ ]:
# 修改为你的 Google Drive 中项目文件夹的路径
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/handwritten-chinese-ocr-samples"

# 复制到 Colab 本地（读写更快）
!cp -r {DRIVE_PROJECT_PATH} /content/
%cd /content/handwritten-chinese-ocr-samples

!ls -la

## 4. 安装依赖

In [ ]:
!pip install -q editdistance
print("依赖安装完成!")

## 5. 预处理 DGRL 数据集

将 CASIA-HWDB2.0 的 `.dgrl` 文件转换为 PNG 图片和标签文件。

**首次运行需要执行此步骤**，之后如果 `data/hwdb2.0` 已存在可跳过。

In [ ]:
import os

# 检查是否已经预处理过
if os.path.exists('data/hwdb2.0/train_img_id_gt.txt'):
    print("数据已预处理，跳过此步骤")
    !wc -l data/hwdb2.0/*_img_id_gt.txt
else:
    print("开始预处理 DGRL 文件...")
    !python preprocess_dgrl.py --train_dir HWDB2.0Train --test_dir HWDB2.0Test --output_dir data/hwdb2.0
    print("\n预处理完成!")

## 6. 检查数据集

In [ ]:
# 查看数据集统计
!echo "=== 数据集统计 ==="
!wc -l data/hwdb2.0/*_img_id_gt.txt

!echo "\n=== 样本示例 ==="
!head -3 data/hwdb2.0/train_img_id_gt.txt

!echo "\n=== 字符数量 ==="
!wc -l data/hwdb2.0/chars_list.txt

In [ ]:
# 可视化几个样本
import matplotlib.pyplot as plt
from PIL import Image
import os

# 读取前3个样本
with open('data/hwdb2.0/train_img_id_gt.txt', 'r', encoding='utf-8') as f:
    samples = [line.strip().split(',', 1) for line in f.readlines()[:3]]

fig, axes = plt.subplots(3, 1, figsize=(15, 6))
for i, (img_name, text) in enumerate(samples):
    img_path = os.path.join('data/hwdb2.0/train', img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(text[:30] + '...' if len(text) > 30 else text)
        axes[i].axis('off')
plt.tight_layout()
plt.show()

## 7. 开始训练

### 参数说明:
- `-m hctr`: 模型类型
- `-d`: 数据集路径
- `-b`: batch size (T4 GPU 建议 8-16)
- `-ep`: 训练轮数
- `-pf`: 打印频率
- `--gpu 0`: 使用 GPU

In [ ]:
# 正式训练 (完整数据集)
!python main.py -m hctr -d data/hwdb2.0 -b 16 -pf 100 -ep 30 --gpu 0

## 8. 测试模型

In [ ]:
# 在测试集上评估
!python test.py -m hctr -f hctr_checkpoint.pth.tar -i data/hwdb2.0 -bm -dm greedy-search --gpu 0

In [ ]:
# 单张图片推理
!python test.py -m hctr -f hctr_checkpoint.pth.tar -i data/hwdb2.0/test/006-P16_000.png -dm greedy-search --gpu 0

## 9. 保存模型到 Google Drive

In [ ]:
# 保存 checkpoint 到 Drive（防止 Colab 断开丢失）
!mkdir -p /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints
!cp hctr_checkpoint.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/
!cp hctr_*acc*.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/ 2>/dev/null || true

# 同时保存预处理好的数据（下次可直接使用）
!cp -r data/hwdb2.0 /content/drive/MyDrive/handwritten-chinese-ocr/

print("模型和数据已保存到 Google Drive!")

## 10. 从断点恢复训练（可选）

如果 Colab 断开，可以从保存的 checkpoint 继续训练：

In [ ]:
# 从 Drive 复制回 checkpoint
# !cp /content/drive/MyDrive/handwritten-chinese-ocr/checkpoints/hctr_checkpoint.pth.tar .

# 恢复训练
# !python main.py -m hctr -d data/hwdb2.0 -b 16 -pf 100 -ep 50 --gpu 0 -re hctr_checkpoint.pth.tar

In [ ]:
# 检查是否分配到 GPU
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. 挂载 Google Drive

将数据集和模型检查点保存到 Google Drive，防止 Colab 断开后丢失。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 创建项目工作目录 (可选)
!mkdir -p /content/drive/MyDrive/handwritten-chinese-ocr

## 3. 获取项目代码

**方式一**: 如果你已将代码上传到 GitHub，直接克隆：
```python
!git clone https://github.com/YOUR_USERNAME/handwritten-chinese-ocr-samples.git
%cd handwritten-chinese-ocr-samples
```

**方式二**: 如果代码在 Google Drive 中，直接复制：

In [ ]:
# 方式二: 从 Google Drive 复制代码 (假设你已将项目文件夹上传到 Drive)
# 请根据你的实际路径修改
!cp -r /content/drive/MyDrive/handwritten-chinese-ocr-samples /content/
%cd /content/handwritten-chinese-ocr-samples

# 查看目录结构
!ls -la

## 4. 安装依赖

In [ ]:
!pip install -q -r requirements.txt
print("依赖安装完成!")

## 5. 准备数据集

### 选项 A: 使用 Demo 数据 (快速测试)
如果只是想验证代码能跑通，运行以下 Cell 生成 demo 数据。

### 选项 B: 使用完整数据集 (正式训练)
1. 下载 CASIA-HWDB 或 SCUT-EPT 数据集
2. 上传到 Google Drive
3. 修改下方路径指向你的数据集

In [ ]:
# 选项 A: 生成 Demo 数据 (用于快速验证)
import os
import shutil

# 创建目录
os.makedirs('data/demo_data/train', exist_ok=True)
os.makedirs('data/demo_data/val', exist_ok=True)
os.makedirs('data/demo_data/test', exist_ok=True)

# 读取样例图片的标签
gt_file = 'images/scut_test_img_id_gt.txt'
samples = []
with open(gt_file, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            parts = line.split(',', 1)
            if len(parts) == 2:
                samples.append((parts[0] + '.jpg', parts[1]))

# 复制图片并生成标签文件
for phase in ['train', 'val', 'test']:
    gt_lines = []
    for img_name, label in samples:
        src = os.path.join('images', img_name)
        dst = os.path.join('data/demo_data', phase, img_name)
        if os.path.exists(src):
            shutil.copy(src, dst)
            gt_lines.append(f"{img_name},{label}")
    
    with open(f'data/demo_data/{phase}_img_id_gt.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(gt_lines))

# 复制字符列表
shutil.copy('data/handwritten_ctr_data/chars_list.txt', 'data/demo_data/chars_list.txt')

print("Demo 数据准备完成!")
!ls -la data/demo_data/

## 6. 开始训练

### 参数说明:
- `-m hctr`: 模型类型 (handwritten Chinese text recognition)
- `-d`: 数据集路径
- `-b`: batch size (根据 GPU 显存调整，T4 建议 8-16)
- `-ep`: 训练轮数
- `-pf`: 打印频率
- `--gpu 0`: 使用第一块 GPU

In [ ]:
# Demo 训练 (快速验证，约 1-2 分钟)
!python main.py -m hctr -d data/demo_data -b 2 -pf 1 -ep 5 --gpu 0

In [ ]:
# 正式训练 (需要完整数据集，可能需要数小时)
# 请将 DATA_PATH 替换为你的实际数据集路径

# DATA_PATH = "/content/drive/MyDrive/your_dataset_folder"
# !python main.py -m hctr -d {DATA_PATH} -b 16 -pf 100 -ep 50 --gpu 0

## 7. 测试模型

In [ ]:
# 使用训练好的模型进行推理
!python test.py -m hctr -f hctr_checkpoint.pth.tar -i images/000000.jpg -dm greedy-search --gpu 0

## 8. 保存模型到 Google Drive

将训练好的模型复制到 Drive，防止 Colab 断开后丢失。

In [ ]:
# 保存最新的 checkpoint 到 Google Drive
!cp hctr_checkpoint.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/
!cp hctr_*.pth.tar /content/drive/MyDrive/handwritten-chinese-ocr/ 2>/dev/null || true
print("模型已保存到 Google Drive!")